0 'PostTypeId',
1 'AcceptedAnswerId',
2 'CreationDate',
3 'Score',
4 'ViewCount',
5 'OwnerUserId',
6 'LastActivityDate',
7 'Tags',
8 'CommentCount',
9 'AnswerCount',
10 'ParentId']

# question - all so tags

In [ ]:
# python的qa的user 在 所有的数据上的 question和answer的tag
import jsonlines
import random
from pickle_file import load_obj, save_obj
from tqdm import tqdm
import json

data_path = 'data_files/task_space_data/'
data_path_save = data_path + 'obj_tag_question_bipartite_core_space/'

question_answer_list = {}
q_tags_bool = {}
with jsonlines.open(f'{data_path}all_so_posts.json', 'r') as fcc_file:
    for line in tqdm(fcc_file):
        k = list(line.keys())[0]
        v = list(line.values())[0]
        q_tags_bool[k] = False
        ##! 限制条件
        if v[0] == '1' and len(k) > 0 and len(v[5]) > 0:
            question_answer_list[k] = []
            q_tags_bool[k] = True

fcc_file.close()


with jsonlines.open(f'{data_path}all_so_posts.json', 'r') as fcc_file:
    for line in tqdm(fcc_file):
        k = list(line.keys())[0]
        v = list(line.values())[0]

        if v[0] == '2' and len(k) > 0 and len(v[5]) > 0 and len(v[10]) > 0 and q_tags_bool[v[10]]:
            question_answer_list[v[10]].append(k)

fcc_file.close()


tag_rename_dict = load_obj('tag_rename_dict', data_path_save)
tagset_all = load_obj('tagset_all', data_path_save)

with jsonlines.open(f'{data_path}all_question_so_tags.json', 'w') as w:
    with jsonlines.open(f'{data_path}all_so_posts.json', 'r') as fcc_file:
        for line in tqdm(fcc_file):
            k = list(line.keys())[0]
            v = list(line.values())[0]
            if q_tags_bool[k]:
                tag_list_temp1 = v[7].lstrip('<').rstrip('>').split('><')
                tag_list_temp = [t.lower().replace('-','_') for t in tag_list_temp1]
                tag_set = set([t for t in tag_list_temp if tag_rename_dict.get(t) is None] + [tag_rename_dict[t] for t in tag_list_temp if tag_rename_dict.get(t) is not None]).intersection(tagset_all)
                if len(tag_set) > 0:
                    w.write({k:[list(tag_set), v[5], v[2], question_answer_list[k]]})

fcc_file.close()

w.close()

# answer - all so tags

In [ ]:
# python的qa的user 在 所有的数据上的 question和answer的tag
import jsonlines
import random
from pickle_file import load_obj, save_obj
from tqdm import tqdm
import json

data_path = 'data_files/task_space_data/'
data_path_save = data_path + 'obj_tag_question_bipartite_core_space/'

##! 初始化
answer_date = {}
answer_user = {}

with jsonlines.open(f'{data_path}all_so_posts.json', 'r') as fcc_file:
    for line in tqdm(fcc_file):
        k = list(line.keys())[0]
        v = list(line.values())[0]
        if v[0] == '2' and len(k) > 0 and len(v[5]) > 0 and len(v[10]) > 0:
            answer_date[k] = v[2]
            answer_user[k] = v[5]

fcc_file.close()

##! 存储answer json文件
with jsonlines.open(f'{data_path}all_question_so_tags.json', 'r') as fcc:
    with jsonlines.open(f'{data_path}all_answer_so_tags.json', 'w') as w:
        for line in tqdm(fcc):
            k = list(line.keys())[0]
            v = list(line.values())[0]
            for ka in v[3]:
                w.write({ka:[v[0], answer_user[ka], answer_date[ka], k]})
                
    w.close()
fcc.close()


# by year

In [ ]:
# 
##! question
import jsonlines
import random
from pickle_file import load_obj, save_obj
from tqdm import tqdm
import json

data_path = 'data_files/task_space_data/'
data_path_save = data_path + 'obj_tag_question_bipartite_core_space/'

for yr in range(2008,2024):
    y = str(yr)
    with jsonlines.open(f'{data_path}all_question_so_tags.json', 'r') as fcc:
        with jsonlines.open(f'{data_path}all_question_so_tags_{yr}.json', 'w') as w:
            for line in tqdm(fcc):
                k = list(line.keys())[0]
                v = list(line.values())[0]
                if v[2][:4] == y:
                    w.write({k:v})
                    
        w.close()
    fcc.close()



##! answer
for yr in range(2008,2024):
    y = str(yr)
    with jsonlines.open(f'{data_path}all_answer_so_tags.json', 'r') as fcc:
        with jsonlines.open(f'{data_path}all_answer_so_tags_{yr}.json', 'w') as w:
            for line in tqdm(fcc):
                k = list(line.keys())[0]
                v = list(line.values())[0]
                if v[2][:4] == y:
                    w.write({k:v})
                    
        w.close()
    fcc.close()


# tag count

In [ ]:
from pickle_file import load_obj, save_obj
import json
import networkx as nx
from tqdm import tqdm
import jsonlines

data_path = 'data_files/task_space_data/'
data_path_save = data_path + 'obj_tag_question_bipartite_core_space/'

tag_list = []

with jsonlines.open(f'{data_path}all_question_so_tags.json', 'r') as fcc_file:
    for line in tqdm(fcc_file):
        k = list(line.keys())[0]
        v = list(line.values())[0]
        tag_list += v[0]

fcc_file.close()

from collections import Counter
tag_count_temp = dict(Counter(tag_list))
tag_count = sorted(tag_count_temp.items(), key = lambda kv:(-kv[1], kv[0]))

save_obj(tag_count,'tag_count_all', data_path)

23234009it [01:12, 319743.74it/s]


In [ ]:
from pickle_file import load_obj, save_obj
import json
import networkx as nx
from tqdm import tqdm
import jsonlines

data_path = 'data_files/task_space_data/'
data_path_save = data_path + 'obj_tag_question_bipartite_core_space/'

tag_count = load_obj('tag_count_all', data_path)
tag_list = [t[0] for t in tag_count]

save_obj(tag_list,'tag_list_all', data_path)


# user list

In [ ]:
# python的qa的user 在 所有的数据上的 question和answer的tag
import jsonlines
import random
from pickle_file import load_obj, save_obj
from tqdm import tqdm
import json

data_path = 'data_files/task_space_data/'
data_path_save = data_path + 'obj_tag_question_bipartite_core_space/'

question_user = {str(yr):[] for yr in range(2008,2024)}
answer_user = {str(yr):[] for yr in range(2008,2024)}

with jsonlines.open(f'{data_path}all_question_so_tags.json', 'r') as fcc_file:
    for line in tqdm(fcc_file):
        k = list(line.keys())[0]
        v = list(line.values())[0]
        question_user[v[2][:4]].append(v[1])
        
fcc_file.close()

with jsonlines.open(f'{data_path}all_answer_so_tags.json', 'r') as fcc_file:
    for line in tqdm(fcc_file):
        k = list(line.keys())[0]
        v = list(line.values())[0]
        answer_user[v[2][:4]].append(v[1])

fcc_file.close()

question_user_temp = {yr:set(ul) for yr, ul in question_user.items()}
answer_user_temp = {yr:set(ul) for yr, ul in answer_user.items()}

save_obj(question_user_temp, 'question_user_by_year', data_path)
save_obj(answer_user_temp, 'answer_user_by_year', data_path)


23234009it [01:14, 311894.77it/s]
33756964it [01:53, 298286.56it/s]


In [ ]:
# python的qa的user 在 所有的数据上的 question和answer的tag
import jsonlines
import random
from pickle_file import load_obj, save_obj
from tqdm import tqdm
import json

data_path = 'data_files/task_space_data/'
data_path_save = data_path + 'obj_tag_question_bipartite_core_space/'

question_list = {str(yr):[] for yr in range(2008,2024)}
answer_list = {str(yr):[] for yr in range(2008,2024)}

with jsonlines.open(f'{data_path}all_question_so_tags.json', 'r') as fcc_file:
    for line in tqdm(fcc_file):
        k = list(line.keys())[0]
        v = list(line.values())[0]
        question_list[v[2][:4]].append(k)
        
fcc_file.close()

with jsonlines.open(f'{data_path}all_answer_so_tags.json', 'r') as fcc_file:
    for line in tqdm(fcc_file):
        k = list(line.keys())[0]
        v = list(line.values())[0]
        answer_list[v[2][:4]].append(k)

fcc_file.close()

save_obj(question_list, 'question_list_by_year', data_path)
save_obj(answer_list, 'answer_list_by_year', data_path)

23234009it [01:15, 307011.32it/s]
33756964it [01:45, 319534.35it/s]


# question and answer bool

In [ ]:
# python的qa的user 在 所有的数据上的 question和answer的tag
import jsonlines
import random
from pickle_file import load_obj, save_obj
from tqdm import tqdm
import json

data_path = 'data_files/task_space_data/'
data_path_save = data_path + 'obj_tag_question_bipartite_core_space/'

question_list = load_obj('question_list_by_year', data_path)
answer_list = load_obj('answer_list_by_year', data_path)

from collections import defaultdict

question_bool = defaultdict(bool)

for ql in tqdm(list(question_list.values())):
    for q in ql:
        question_bool[q] = True


answer_bool = defaultdict(bool)

for ql in tqdm(list(answer_list.values())):
    for q in ql:
        answer_bool[q] = True

save_obj(question_bool, 'question_bool_dict', data_path)
save_obj(answer_bool, 'answer_bool_dict', data_path)


100%|██████████| 16/16 [00:12<00:00,  1.27it/s]


# question_answer_list

In [ ]:
# python的qa的user 在 所有的数据上的 question和answer的tag
import jsonlines
import random
from pickle_file import load_obj, save_obj
from tqdm import tqdm
import json

data_path = 'data_files/task_space_data/'
data_path_save = data_path + 'obj_tag_question_bipartite_core_space/'

question_answer_list = {}

with jsonlines.open(f'{data_path}all_question_so_tags.json', 'r') as fcc_file:
    for line in tqdm(fcc_file):
        k = list(line.keys())[0]
        v = list(line.values())[0]
        question_answer_list[k] = len(v[3])

fcc_file.close()

save_obj(question_answer_list, 'question_answer_list', data_path)


23234009it [01:29, 258379.11it/s]


In [ ]:
# python的qa的user 在 所有的数据上的 question和answer的tag
import jsonlines
import random
from pickle_file import load_obj, save_obj
from tqdm import tqdm
import json

data_path = 'data_files/task_space_data/'
data_path_save = data_path + 'obj_tag_question_bipartite_core_space/'

answer_question_dict = {}

with jsonlines.open(f'{data_path}all_answer_so_tags.json', 'r') as fcc_file:
    for line in tqdm(fcc_file):
        k = list(line.keys())[0]
        v = list(line.values())[0]
        answer_question_dict[k] = v[3]

fcc_file.close()

save_obj(answer_question_dict, 'answer_question_dict', data_path)


33756964it [02:17, 245313.73it/s]


# question date and answer date

In [ ]:
# python的qa的user 在 所有的数据上的 question和answer的tag
import jsonlines
import random
from pickle_file import load_obj, save_obj
from tqdm import tqdm
import json

data_path = 'data_files/task_space_data/'
data_path_save = data_path + 'obj_tag_question_bipartite_core_space/'

question_date = {}

with jsonlines.open(f'{data_path}all_question_so_tags.json', 'r') as fcc_file:
    for line in tqdm(fcc_file):
        k = list(line.keys())[0]
        v = list(line.values())[0]
        question_date[k] = v[2]

fcc_file.close()

save_obj(question_date, 'question_date', data_path)

del question_date

answer_date = {}

with jsonlines.open(f'{data_path}all_answer_so_tags.json', 'r') as fcc_file:
    for line in tqdm(fcc_file):
        k = list(line.keys())[0]
        v = list(line.values())[0]
        answer_date[k] = v[2]

fcc_file.close()

save_obj(answer_date, 'answer_date', data_path)


23234009it [01:22, 281188.61it/s]
33756964it [02:02, 275543.93it/s]


# user answer time hour

In [ ]:
import jsonlines
import random
from pickle_file import load_obj, save_obj
from tqdm import tqdm
import json

data_path = 'data_files/task_space_data/'
data_path_save = data_path + 'obj_tag_question_bipartite_core_space/'

so_year_list = [yr for yr in range(2008,2024)]
user_answer_time = {}

for yr in so_year_list:
    print(yr)
    with jsonlines.open(f'{data_path}all_answer_so_tags_{str(yr)}.json', 'r') as fcc_file:
        for line in tqdm(fcc_file):
            v = list(line.values())[0]
            if user_answer_time.get(v[1]) is None:
                user_answer_time[v[1]] = [0 for h in range(24)]

            user_answer_time[v[1]][int(v[2][11:13])] += 1

    fcc_file.close()

save_obj(user_answer_time,'user_answer_time_hour', data_path)


2008


184951it [00:00, 288107.30it/s]


2009


813876it [00:02, 282235.84it/s]


2010


...

1939169it [00:10, 181747.85it/s]


2022


1695331it [00:08, 208118.29it/s]


2023


536547it [00:02, 235558.56it/s]
